# Romberg integration
[Romberg’s method](https://en.wikipedia.org/wiki/Romberg%27s_method) to find a definite integral combines two formulas, [extended trapezoidal](https://en.wikipedia.org/wiki/Trapezoidal_rule) rule and [Richardson extrapolation](https://en.wikipedia.org/wiki/Richardson_extrapolation), to get a good approximation in relatively low number of steps.

Let’s explain how it all works without causing a serious headache.

The simplest way to find a definite integral of function **f** on interval **(a, b)** is to use a trapezoidal rule.

$\int_a^bf(x)dx \approx \frac{b-a}{2}(f(a)+f(b))$

trapezoidal rule

It’s the left-most chart and you can see that the formula is merely an area of [trapezoid](https://en.wikipedia.org/wiki/Trapezoid).

![day98-romberg_integration_1](resource/day98-romberg_integration_1.png)

definite integral estimates

You can also notice that there’s a certain error which can be improved using extended trapezoidal rule. Split interval **(a, b)** into two halves, apply trapezoidal rule onto each half and sum them up.

$\int_a^bf(x)dx \approx \frac{b-a}{4}(f(a)+2f(\frac{a+b}{2}+f(b)))$

extended trapezoidal rule

It’s the chart in the middle. The formula is merely an area of two trapezoids.

The third plot contains another halving and even better estimate. We can go on and on until we are satisfied with the estimate error.

The problem is, the convergence in this way would be very slow. And here comes Richardson extrapolation to the rescue.

In Richardson extrapolation we take two bad estimates and combine them into a good one. The best explanation I’ve ever seen is definitely described in this Dilbert strip, check it out.

![day96-romber_integration_2](resource/day96-romber_integration_2.gif)

Dilbert Comic Strip on 2008-05-07 | Dilbert by Scott Adams

However, unlike Dilbert we are lucky since we know error bound of each estimate.

In other words, if we are sure that one estimate is better that the other [e.g. due to smaller interval in extended trapezoidal rule], we also know in which direction should we expect the exact solution.

$F(0) = \frac{2^kF(\frac{h}{2})-F(h)}{2^k-1}+O(h^{k+1})$

Richardson extrapolation

Finally, Romberg’s method builds a table of estimates with surprisingly accurate results.

![day98-romberg_integration_2](resource/day98-romberg_integration_2.png)

Romberg integration

The first column contains results of extended trapezoidal rule, each row using twice as many splits as the row before. All the other values are results of Richardson extrapolation.

The diagonal contains the final estimates and each diagonal element gives about **two more digits of precision**. Hence the element at 5th row and 5th column gives an estimate of about 10 digits of accuracy.

In [1]:
import numpy as np
import sys

np.set_printoptions(precision=14, linewidth=120)

## algorithm

In [2]:
def integrate(fn, a, b, steps=5, debug=False, exact=None):
    table = np.zeros((steps, steps), dtype=np.float64)
    pow_4 = 4 ** np.arange(steps, dtype=np.float64) - 1

    # trapezoidal rule
    h = (b - a)
    table[0, 0] = h * (fn(a) + fn(b)) / 2

    for j in range(1, steps):
        h /= 2

        # extended trapezoidal rule
        table[j, 0] = table[j - 1, 0] / 2
        table[j, 0] += h * np.sum(fn(a + i * h) for i in range(1, 2 ** j + 1, 2))

        # richardson extrapolation
        for k in range(1, j + 1):
            table[j, k] = table[j, k - 1] + (table[j, k - 1] - table[j - 1, k - 1]) / pow_4[k]

    # debug
    if debug:
        print(table, file=sys.stderr)
        if exact is not None:
            errors = ['%.2e' % i for i in np.abs(table.diagonal() - exact)]
            print('abs. error:', errors, file=sys.stderr)

    return table[-1, -1]

## integration

In [3]:
# integral[0, 1] of e^(-x^2)
integrate(lambda x: np.exp(-x * x), 0, 1, debug=True, exact=0.746824132812427)

[[0.68393972058572 0.               0.               0.               0.              ]
 [0.73137025182856 0.74718042890951 0.               0.               0.              ]
 [0.74298409780038 0.74685537979099 0.74683370984975 0.               0.              ]
 [0.7458656148457  0.74682612052747 0.7468241699099  0.74682401848228 0.              ]
 [0.74658459678822 0.74682425743573 0.74682413322961 0.74682413264739 0.74682413309509]]
abs. error: ['6.29e-02', '3.56e-04', '9.58e-06', '1.14e-07', '2.83e-10']


0.7468241330950943

In [4]:
# ln(2)
integrate(1..__truediv__, 1, 2, debug=True, exact=np.log(2))

[[0.75             0.               0.               0.               0.              ]
 [0.70833333333333 0.69444444444444 0.               0.               0.              ]
 [0.69702380952381 0.69325396825397 0.6931746031746  0.               0.              ]
 [0.69412185037185 0.69315453065453 0.69314790148123 0.69314747764483 0.              ]
 [0.69339120220753 0.69314765281942 0.69314719429708 0.69314718307193 0.69314718191674]]
abs. error: ['5.69e-02', '1.30e-03', '2.74e-05', '2.97e-07', '1.36e-09']


0.693147181916745

In [5]:
# integral[0, 1] of x^3
integrate(lambda x: x**3, 0, 1, debug=True, exact=.25)

[[0.5          0.           0.           0.           0.          ]
 [0.3125       0.25         0.           0.           0.          ]
 [0.265625     0.25         0.25         0.           0.          ]
 [0.25390625   0.25         0.25         0.25         0.          ]
 [0.2509765625 0.25         0.25         0.25         0.25        ]]
abs. error: ['2.50e-01', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']


0.25

## logarithmus naturalis

In [6]:
def ln(x):
    if x <= 0:
        raise ValueError()
    m, e = np.frexp(x)
    return integrate(1..__truediv__, 1, m) + e * 0.6931471805599453

In [7]:
ln(np.e)

0.999999999997895

In [8]:
ln(2)

0.6931471792031456

In [9]:
ln(np.pi)

1.1447298858493882

## normal distribution

In [10]:
def norm_pdf(x, mean, sd):
    x0 = (x - mean) ** 2
    v2 = 2 * sd ** 2
    return np.exp(-x0 / v2) / np.sqrt(np.pi * v2)

In [11]:
def norm_cdf(x, mean, sd):
    return integrate(lambda x: norm_pdf(x, mean, sd), mean, x) + .5

In [12]:
norm_cdf(1.96, mean=0, sd=1)

0.9750021118942748

In [13]:
norm_cdf(1.2, mean=1, sd=.2) - norm_cdf(0.8, mean=1, sd=.2)

0.6826894921375496